In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [16]:
# !pip install langchain-teddynote

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH99-MyTest")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH99-MyTest


In [3]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from chromadb.config import Settings
from tqdm import tqdm
import os

In [4]:
# 현재 디렉토리에 DB 저장 경로 설정
current_dir = os.getcwd()
db_path = os.path.join(current_dir, "my_db.db")

# ChromaDB 클라이언트 초기화 (영구 저장소 설정)
client = chromadb.PersistentClient(
    path=db_path, settings=Settings(allow_reset=True, is_persistent=True)
)

In [5]:
# Embedding 함수 설정
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)


def reset_collection(client, collection_name):
    try:
        # 기존 컬렉션 존재 여부 확인
        existing_collections = client.list_collections()
        collection_exists = any(
            col.name == collection_name for col in existing_collections
        )

        if collection_exists:
            print(f"기존 컬렉션 '{collection_name}'을 삭제합니다.")
            client.delete_collection(collection_name)
            print(f"컬렉션 '{collection_name}'이 삭제되었습니다.")

        # 새로운 컬렉션 생성
        print(f"새로운 컬렉션 '{collection_name}'을 생성합니다.")
        collection = client.create_collection(
            name=collection_name, embedding_function=embedding_function
        )

        print(f"컬렉션 '{collection_name}'이 성공적으로 초기화되었습니다.")
        return collection

    except Exception as e:
        print(f"컬렉션 초기화 중 오류 발생: {str(e)}")
        raise


# 컬렉션 초기화 실행
collection = reset_collection(client, "my_collection")

# 컬렉션 정보 확인
print("\n=== 초기화된 컬렉션 정보 ===")
print(f"컬렉션 이름: {collection.name}")
print(f"컬렉션 크기: {collection.count()} 문서")

/Users/dubutan/Library/Caches/pypoetry/virtualenvs/langchain-kr-uRT90lmq-py3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/dubutan/Library/Caches/pypoetry/virtualenvs/langchain-kr-uRT90lmq-py3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


기존 컬렉션 'my_collection'을 삭제합니다.
컬렉션 'my_collection'이 삭제되었습니다.
새로운 컬렉션 'my_collection'을 생성합니다.
컬렉션 'my_collection'이 성공적으로 초기화되었습니다.

=== 초기화된 컬렉션 정보 ===
컬렉션 이름: my_collection
컬렉션 크기: 0 문서


In [112]:
# 컬렉션 생성 (이미 존재하는 경우 가져오기)
# try:
#     collection = client.get_or_create_collection(
#         name="my_collection", embedding_function=embedding_function
#     )
# except ValueError:
#     # 이미 존재하는 경우 기존 컬렉션 가져오기
#     collection = client.get_collection(
#         name="my_collection", embedding_function=embedding_function
#     )

In [6]:
# DataFrame에서 ChromaDB용 데이터 준비 및 저장
def prepare_and_store_data(df, collection):
    # 배치 크기 설정
    BATCH_SIZE = 100
    total_rows = len(df)  # min(3000, len(df))  # 최대 3000개 row로 제한

    documents = []
    metadatas = []
    ids = []

    # tqdm으로 진행률 표시
    for idx in tqdm(range(total_rows)):
        row = df.iloc[idx]

        # 공고명(col4)과 공고내용(col16)을 document content로 저장
        documents.append(
            str(row["col4"]) + " " + str(row["col15"]) + " " + str(row["col25"])
        )

        # title, category, date를 metadata로 저장
        metadatas.append(
            {
                "support_area": str(row["col5"]),  # 지원분야
                "area": str(row["col6"]),  # 지역
                "target": str(row["col7"]),  # 지원대상
                "age": str(row["col8"]),  # 대상연령
                "work_history": str(row["col10"]),  # 업력
                "organizaion": str(row["col11"]),  # 기관명
                "orag_class": str(row["col12"]),  # 기관구분 : 공공, 민간, 교육
                "department": str(row["col13"]),  # 담당부서
                "apply_visit": str(row["col19"]),  # 신청방법 : 방문
                "apply_mail": str(row["col20"]),  # 신청방법 : 우편
                "apply_fax": str(row["col21"]),  # 신청방법 : FAX
                "apply_email": str(row["col22"]),  # 신청방법 : email
                "apply_online": str(row["col23"]),  # 신청방법 : 온라인
                "apply_target": str(row["col25"]),  # 신청대상
                "apply_exlude": str(row["col20"]),  # 제외대상
            }
        )

        ids.append(str(idx))

        # 배치 크기에 도달하면 저장
        if len(documents) == BATCH_SIZE or idx == total_rows - 1:
            print(f"\n배치 저장 중... ({len(documents)} 문서)")
            collection.add(documents=documents, metadatas=metadatas, ids=ids)

            # 배치 초기화
            documents = []
            metadatas = []
            ids = []

    return total_rows

In [7]:
# CSV 파일 읽기
df = pd.read_csv("./data/gong.csv")

In [8]:
print(len(df))

3150


In [ ]:
# collection = get_or_create_collection(client, "my_collection")

total_documents = prepare_and_store_data(df, collection)
print(f"\n총 {total_documents}개의 문서가 성공적으로 저장되었습니다.")

In [10]:
# 저장된 데이터 샘플 확인
print("\n=== 저장된 데이터 샘플 확인 ===")
sample_results = collection.get(ids=["0"], include=["documents", "metadatas"])
print("\nDocument:", sample_results["documents"][0])
print("Metadata:", sample_results["metadatas"][0])

# 컬렉션 정보 출력
print(f"\n현재 컬렉션의 총 문서 수: {collection.count()}")


=== 저장된 데이터 샘플 확인 ===

Document: [인천대학교 창업지원단] 2024년 예비창업패키지 대비 INU 스타트업 아카데미 교육생 모집 2024년 예비창업패키지를 준비하는 예비창업자를 발굴하고 지원하고자 참여를 희망하는 예비창업자는 안내에 따라 신청하기 바랍니다. nan
Metadata: {'age': '전체', 'apply_email': 'nan', 'apply_exlude': 'nan', 'apply_fax': 'nan', 'apply_mail': 'nan', 'apply_online': 'nan', 'apply_target': 'nan', 'apply_visit': 'nan', 'area': '전국', 'department': '창업지원단', 'orag_class': '교육기관', 'organizaion': '국립대학법인 인천대학교', 'support_area': '사업화', 'target': '전체', 'work_history': '예비창업자'}

현재 컬렉션의 총 문서 수: 3150


In [11]:
# 첫 번째 document 가져오기
result = collection.get(
    ids=["0"],  # 첫 번째 행의 ID
    include=["documents", "embeddings"],  # documents와 embeddings 모두 포함
)

print("Document 내용:")
print(result["documents"][0])  # 첫 번째 document의 텍스트 내용

print("\nEmbedding 벡터 (처음 10개 값):")
print(result["embeddings"][0][:100])  # 임베딩 벡터의 처음 10개 값만 출력

Document 내용:
[인천대학교 창업지원단] 2024년 예비창업패키지 대비 INU 스타트업 아카데미 교육생 모집 2024년 예비창업패키지를 준비하는 예비창업자를 발굴하고 지원하고자 참여를 희망하는 예비창업자는 안내에 따라 신청하기 바랍니다. nan

Embedding 벡터 (처음 10개 값):
[0.024758411571383476, 0.11222247779369354, 0.10782627016305923, -0.07709864526987076, -0.06997625529766083, -0.03515668585896492, 0.05440877750515938, 0.01205610390752554, -0.009746531955897808, 0.016725560650229454, 0.09892278909683228, -0.07387170195579529, 0.10022801160812378, -0.0635632798075676, 0.011222665198147297, -0.08720552176237106, 0.05799970030784607, -0.009464068338274956, -0.08313225954771042, 0.008907158859074116, 0.038746219128370285, 0.009589043445885181, -0.003916285000741482, 0.030059946700930595, -0.08925820887088776, -0.0009496996062807739, -0.04574006050825119, -0.03999411314725876, 0.02950669452548027, 0.11263961344957352, -0.03961150720715523, 0.10477109253406525, 0.006684944033622742, -0.0001435567537555471, -0.08199677616357803, -0.017175424844026566, 0.0053360178135335445, 0.0636972188949585, 

In [ ]:
from langchain_openai import ChatOpenAI
import chromadb
from chromadb.utils import embedding_functions
import os

# 쿼리 실행
query = "지원 지역이 서울특별시인 지원사업은"
# results = collection.query(query_texts=[query], n_results=3)  # 상위 3개 결과 가져오기
results = collection.query(
    query_texts=[query],
    n_results=10,
    # where={
    #     "area" : "서울특별시",
    # }
)

results["documents"]

In [ ]:
for i, doc in enumerate(results["documents"][0]):
    print(f"\nDocument {i+1}:")
    print(f"Content: {doc}")
    print(f"Metadata: {results['metadatas'][0][i]}")

In [ ]:
# OpenAI를 사용한 답변 생성
# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
)

# 프롬프트 구성
prompt = f"""다음은 예비창업자가 지원할 수 있는 지원사업에 대한 검색 결과입니다. 
이를 바탕으로 예비창업자가 지원할 수 있는 주요 지원사업들을 간단명료하게 요약해서 설명해주세요:

{results['documents'][0]}
"""

# ChatGPT API 호출
# response = client.chat.completions.create(
#     model="gpt-4o",
#     messages=[
#         {
#             "role": "system",
#             "content": "You are a helpful assistant that explains startup support programs in Korean.",
#         },
#         {"role": "user", "content": prompt},
#     ],
#     temperature=0,
#     max_tokens=1000,
# )
response = llm.invoke(prompt)

print("\n=== AI 답변 ===")
print(response.content)